In [1]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent  # since notebook is in ./test_env
sys.path.insert(0, str(ROOT))

In [2]:
import traceback
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr

from backend.fx_sr.config import FXConfig
from backend.fx_sr.data_loader import MarketDataLoader
from backend.fx_sr.features import FeatureEngine
from backend.fx_sr.schemas import BeliefParams
from backend.fx_sr.sr import SREngine
from backend.fx_sr.transitions import TransitionModel

In [3]:
class ComprehensiveAuditor:
    def __init__(self):
        self.config = FXConfig()
        self.loader = MarketDataLoader(self.config)
        self.features = FeatureEngine()
        self.physics = TransitionModel(self.config)
        self.math = SREngine(self.config)
        self.currencies = list(self.config.UNIVERSE.keys())

    def run(self):
        print("\n" + "="*60)
        print("INITIALIZING ROBUST STRATEGY AUDIT")
        print("="*60)

        # 1. Load Data
        try:
            df = self.loader.fetch_history(lookback_days=1500)
            feat_dict, _ = self.features.compute_features(df, self.currencies)
            macro_df = self.features.compute_macro_regime_indices(df, self.currencies)
            
            yields = self.loader.fetch_yields(df.index)
            yield_diffs = self.features.compute_yield_differentials(yields, self.currencies)
            vix_z_series = self.features.compute_adaptive_tuning(df['VIX'])
        except Exception as e:
            print(f"FAILED TO LOAD DATA: {e}")
            return

        # 2. Setup Simulation
        test_dates = df.index[252:-65:5] 
        if len(test_dates) == 0:
            print("ERROR: Not enough data for the requested lookback window.")
            return

        records = []
        print(f"Simulating {len(test_dates)} events...")

        for date in test_dates:
            try:
                # Inputs
                mom = feat_dict["mom_21d"].loc[date].fillna(0)
                vol = feat_dict["volatility"].loc[date].fillna(0)
                y_diff = yield_diffs.loc[date].fillna(0)
                vix_z = vix_z_series.loc[date]
                idx_row = macro_df.loc[date]

                # Physics
                T_base = self.physics.construct_physics_matrix(mom, vol, y_diff, BeliefParams(), vix_z)
                T_adj, leakage, net_flow, regime = self.physics.apply_adaptive_leakage(
                    T_base, self.currencies, vix_z, idx_row
                )

                # Solve SR (Medium Horizon - 63 Days)
                gamma = self.math.get_gamma(63)
                M = self.math.compute_sr_matrix(T_adj, gamma)
                scores = self.math.compute_strength_scores(M)
                
                ranks = np.argsort(scores)[::-1]
                top_picks = [self.currencies[i] for i in ranks[:2]]
                btm_picks = [self.currencies[i] for i in ranks[-2:]]

                # Look Forward
                curr_idx = df.index.get_loc(date)
                fut_idx = curr_idx + 63 
                
                future_date = df.index[fut_idx]
                fwd_rets = (df.loc[future_date, self.currencies] - df.loc[date, self.currencies]) / df.loc[date, self.currencies]
                
                ic, _ = spearmanr(scores, fwd_rets.values)
                strategy_ret = fwd_rets[top_picks].mean() - fwd_rets[btm_picks].mean()
                
                records.append({
                    'date': date,
                    'regime': regime.label,
                    'ic': ic,
                    'strategy_return': strategy_ret,
                    'top_pick': top_picks[0]
                })

            except Exception as e:
                # UNCOMMENT the next line if you still get 0 observations to see the error
                # print(f"Skipping {date}: {e}")
                continue

        # 3. Aggregation
        if not records:
            print("CRITICAL ERROR: No observations were recorded. Every day failed.")
            return

        results = pd.DataFrame(records)
        self.generate_report(results)

    def generate_report(self, df):
        print("\n" + "-"*30)
        print("EXECUTIVE SUMMARY")
        print("-"*30)
        print(f"Observations:     {len(df)}")
        print(f"Mean Rank IC:     {df['ic'].mean():.4f}")
        print(f"Win Rate (IC>0):  {(df['ic'] > 0).mean()*100:.1f}%")
        print(f"Cum. L/S Return:  {df['strategy_return'].sum()*100:.1f}%")

        print("\n" + "-"*30)
        print("PERFORMANCE BY REGIME")
        print("-"*30)
        reg_perf = df.groupby('regime').agg({'ic': 'mean', 'strategy_return': 'mean'})
        reg_perf['strategy_return'] *= 100
        print(reg_perf.rename(columns={'strategy_return': 'Avg Ret %'}))

        print("\n" + "-"*30)
        print("ASSET DIVERSITY")
        print("-"*30)
        print(df['top_pick'].value_counts(normalize=True).head(5))

In [4]:
if __name__ == "__main__":
    auditor = ComprehensiveAuditor()
    auditor.run()


INITIALIZING ROBUST STRATEGY AUDIT
Fetching 12 tickers ['EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X', 'JPY=X', 'CHF=X', 'CAD=X', '^VIX', 'DX-Y.NYB', 'GC=F', 'SPY', '^TNX'] from Yahoo Finance...
Simulating 248 events...

------------------------------
EXECUTIVE SUMMARY
------------------------------
Observations:     248
Mean Rank IC:     0.0894
Win Rate (IC>0):  60.1%
Cum. L/S Return:  75.1%

------------------------------
PERFORMANCE BY REGIME
------------------------------
                           ic  Avg Ret %
regime                                  
Neutral              0.140236   0.670510
Reflation / Risk-On  0.043177   0.128058
Tightening / Carry   0.072581   0.053634
US-Centric Stress    0.044643  -0.907375
USD Wrecking Ball   -0.113095  -1.035881

------------------------------
ASSET DIVERSITY
------------------------------
top_pick
CAD    0.629032
GBP    0.100806
JPY    0.100806
EUR    0.100806
CHF    0.064516
Name: proportion, dtype: float64
